In [1]:
import rebound
import time
import sys
import numpy as np
from matplotlib import pyplot as plt
from astropy import units as u
from astropy import constants as c

# local imports
import heartbeat
import globs
import metasim

In [2]:
def run_sim(filestem='test/test'):
    
    # global variables

    globs.initialise()

    globs.glob_dclo = 1. #CE distance to check in rH
    #dir = 'test/'
    globs.glob_archive = ''
    #globals.glob_names = []
    globs.glob_is_close = False

    system = metasim.MetaSim(filestem=filestem)
    system.run_planets()
    if globs.glob_is_close:
        system.rewind()
        system.add_moons()
        system.run_moons()
        system.analyse()
        system.make_timeline()

In [10]:
#Nsys = 1

#for i in range(Nsys):
#    run_sim(filestem=f'test/test{i:04d}')

i = 13
run_sim(filestem=f'test/test{i:04d}')

Simulation test/test0013

Creating new simulation...
Running planets-only simulation...
A particle escaped (r>exit_max_distance).
Planets-only simulation complete
171299.92499232147 years took 1049.8890681266785 seconds
